In [14]:
import pandas as pd
import geopandas as gpd
import h5py
import glob
import os
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, LinearColorMapper

import numpy as np
from ipyleaflet import Map, basemaps, basemap_to_tiles, GeoData, LayersControl
import geopandas
import json

In [15]:

hdf5_flist = glob.glob('./Outputs/*.h5')
f = h5py.File(hdf5_flist[0], 'r')

In [16]:
hdf5_flist

['./Outputs/processed_ATL06_20181231145919_00480211_001_01.h5']

In [17]:
f.keys()

"""create empty beam specific dataframes"""
gtl1_df = pd.DataFrame(columns = ['delta_time','lat','lon','h_li','x_atc', 'atl06_quality_summary', 'bsnow_conf'])
gtr1_df = pd.DataFrame(columns = ['delta_time','lat','lon','h_li','x_atc', 'atl06_quality_summary', 'bsnow_conf'])
gtl2_df = pd.DataFrame(columns = ['delta_time','lat','lon','h_li','x_atc', 'atl06_quality_summary', 'bsnow_conf'])
gtr2_df = pd.DataFrame(columns = ['delta_time','lat','lon','h_li','x_atc', 'atl06_quality_summary', 'bsnow_conf'])
gtl3_df = pd.DataFrame(columns = ['delta_time','lat','lon','h_li','x_atc', 'atl06_quality_summary', 'bsnow_conf'])
gtr3_df = pd.DataFrame(columns = ['delta_time','lat','lon','h_li','x_atc', 'atl06_quality_summary', 'bsnow_conf'])

In [18]:
gt1l = f['gt1l']
gt1r = f['gt1r']
gt2l = f['gt2l']
gt2r = f['gt2r']
gt3l = f['gt3l']
gt3r = f['gt3r']
iceseg_gtl1 = gt1l['land_ice_segments']
iceseg_gt1r = gt1r['land_ice_segments']
iceseg_gt21 = gt2l['land_ice_segments']
iceseg_gt2r = gt2r['land_ice_segments']
iceseg_gt31 = gt3l['land_ice_segments']
iceseg_gt3r = gt3r['land_ice_segments']



"""adding beam specific time to df's"""
gtl1_df['delta_time'] = iceseg_gtl1['delta_time']
gtr1_df['delta_time'] = iceseg_gt1r['delta_time']
gtl2_df['delta_time'] = iceseg_gt21['delta_time']
gtr2_df['delta_time'] = iceseg_gt2r['delta_time']
gtl3_df['delta_time'] = iceseg_gt31['delta_time']
gtr3_df['delta_time'] = iceseg_gt3r['delta_time']


"""adding beam specific lat to df's"""
gtl1_df['lat'] = iceseg_gtl1['latitude']
gtr1_df['lat'] = iceseg_gt1r['latitude']
gtl2_df['lat'] = iceseg_gt21['latitude']
gtr2_df['lat'] = iceseg_gt2r['latitude']
gtl3_df['lat'] = iceseg_gt31['latitude']
gtr3_df['lat'] = iceseg_gt3r['latitude']

"""adding beam specific lon to df's"""
gtl1_df['lon'] = iceseg_gtl1['longitude']
gtr1_df['lon'] = iceseg_gt1r['longitude']
gtl2_df['lon'] = iceseg_gt21['longitude']
gtr2_df['lon'] = iceseg_gt2r['longitude']
gtl3_df['lon'] = iceseg_gt31['longitude']
gtr3_df['lon'] = iceseg_gt3r['longitude']

"""adding beam specific h_li to df's"""
gtl1_df['h_li'] = iceseg_gtl1['h_li']
gtr1_df['h_li'] = iceseg_gt1r['h_li']
gtl2_df['h_li'] = iceseg_gt21['h_li']
gtr2_df['h_li'] = iceseg_gt2r['h_li']
gtl3_df['h_li'] = iceseg_gt31['h_li']
gtr3_df['h_li'] = iceseg_gt3r['h_li']

"""adding beam specific x_atc to df's"""
gtl1_df['x_atc'] = iceseg_gtl1['ground_track']['x_atc']
gtr1_df['x_atc'] = iceseg_gt1r['ground_track']['x_atc']
gtl2_df['x_atc'] = iceseg_gt21['ground_track']['x_atc']
gtr2_df['x_atc'] = iceseg_gt2r['ground_track']['x_atc']
gtl3_df['x_atc'] = iceseg_gt31['ground_track']['x_atc']
gtr3_df['x_atc'] = iceseg_gt3r['ground_track']['x_atc']

"""adding quality flag to df's"""
gtl1_df['atl06_quality_summary'] = iceseg_gtl1['atl06_quality_summary']
gtr1_df['atl06_quality_summary'] = iceseg_gt1r['atl06_quality_summary']
gtl2_df['atl06_quality_summary'] = iceseg_gt21['atl06_quality_summary']
gtr2_df['atl06_quality_summary'] = iceseg_gt2r['atl06_quality_summary']
gtl3_df['atl06_quality_summary'] = iceseg_gt31['atl06_quality_summary']
gtr3_df['atl06_quality_summary'] = iceseg_gt3r['atl06_quality_summary']


gtl1_df['bsnow_conf'] = iceseg_gtl1['geophysical']['bsnow_conf']
gtr1_df['bsnow_conf'] = iceseg_gt1r['geophysical']['bsnow_conf']
gtl2_df['bsnow_conf'] = iceseg_gt21['geophysical']['bsnow_conf']
gtr2_df['bsnow_conf'] = iceseg_gt2r['geophysical']['bsnow_conf']
gtl3_df['bsnow_conf'] = iceseg_gt31['geophysical']['bsnow_conf']
gtr3_df['bsnow_conf'] = iceseg_gt3r['geophysical']['bsnow_conf']

In [19]:
beam1_df = pd.concat([gtl1_df, gtr1_df], axis=1)
beam2_df = pd.concat([gtl2_df, gtr2_df], axis=1)
beam3_df = pd.concat([gtl3_df, gtr3_df], axis=1)

In [20]:
gtl1_df = gtl1_df[gtl1_df['atl06_quality_summary'] == 0]
gtr1_df = gtr1_df[gtr1_df['atl06_quality_summary'] == 0]

In [23]:
gtl1_df= gtl1_df[gtl1_df['x_atc'] < 28975107]
gtr1_df = gtr1_df[gtr1_df['x_atc'] < 28975107]
# # create a new plot with a title and axis labels
y_min, y_max =(np.min(gtl1_df['h_li'])-2),(np.max(gtl1_df['h_li'])+2)

p = figure(title="atl06", x_axis_label='x_atc', y_axis_label='h_li', y_range=(y_min,y_max),sizing_mode='stretch_both')

# add a line renderer with legend and line thickness
# p.line(gtl1_df['x_atc'], gtl1_df['h_li'], legend="Temp.", line_width=2)

color_list = ['crimson','darkred','cornflowerblue','darkblue','mediumseagreen','darkgreen']


""" add shaded areas where blowing snow == 'high'"""

p.vbar(x=gtr1_df['x_atc'][gtr1_df['bsnow_conf']==6], width=5, bottom=0,
       top=y_max, color="coral", alpha=0.5)
  
       
p.scatter(gtl1_df['x_atc'], gtl1_df['h_li'],color = color_list[0], size=10, fill_alpha=0.5,legend='gtl1')
p.scatter(gtr1_df['x_atc'], gtr1_df['h_li'], color = color_list[1],size=10, fill_alpha=0.5,legend='gtr1')
p.scatter(gtl2_df['x_atc'], gtl1_df['h_li'], color = color_list[2],size=10, fill_alpha=0.5,legend='gtl2')
p.scatter(gtr2_df['x_atc'], gtr1_df['h_li'], color = color_list[3],size=10, fill_alpha=0.5,legend='gtr2')
p.scatter(gtl3_df['x_atc'], gtl1_df['h_li'], color = color_list[4],size=10, fill_alpha=0.5,legend='gtl3')
p.scatter(gtr3_df['x_atc'], gtr1_df['h_li'], color = color_list[5],size=10, fill_alpha=0.5,legend='gtr3')
# show the results
p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)
save(p, 'megadunes.html')


# from bokeh.plotting import figure, output_file, save
# p = figure(title="Basic Title", plot_width=300, plot_height=300)
# p.circle([1, 2], [3, 4])
# output_file("test.html")
# save(p)

'/home/nrhagen/Documents/sathackweek/project/WaterNoice/notebooks/megadunes.html'

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
f = plt.figure()
ax3D = f.add_subplot(111, projection='3d')
# ax3D.scatter(gtl1_df['lon'], gtl1_df['lat'], gtl1_df['h_li'], c=gtl1_df['bsnow_conf'], s=1, cmap='inferno')
ax3D.scatter(gtr1_df['lon'], gtr1_df['lat'], gtr1_df['h_li'], c=gtr1_df['bsnow_conf'], s=1, cmap='inferno')
# gtl1_df = ax3D.scatter(gtl1_df['lon'], gtl1_df['lat'], gtl1_df['h_li'], c=gtl1_df['bsnow_conf'], s=1, cmap='inferno')
# gtl1_df = ax3D.scatter(gtl1_df['lon'], gtl1_df['lat'], gtl1_df['h_li'], c=gtl1_df['bsnow_conf'], s=1, cmap='inferno')

#sc = ax3D.scatter(glas_df['lon'], glas_df['lat'], glas_df['glas_z'], c=glas_df['decyear'], s=1, cmap='inferno')
plt.colorbar(sc)

In [ ]:
gtl1_df

In [10]:
# import fiona

# fiona.supported_drivers['KML'] = 'rw'

# gtl1_gdf = geopandas.GeoDataFrame(
#     gtl1_df, geometry=geopandas.points_from_xy(gtl1_df.lon, gtl1_df.lat))

# # gtl1_gdf.to_file("gtl1_gdf_dunes.gpkg", driver="GPKG")
# gtl1_gdf.to_file('gtl1_gdf_dunes.kml', driver='KML')


In [ ]:
"""Blowing snow confidence. -3=surface not detected; -2=no
surface wind;-1=no scattering layer found; 0=no top layer
found; 1=none­little; 2=weak; 3=moderate; 4=moderate­high;
5=high; 6=very high
flag_values: -3, -2, -1, 0, 1, 2, 3, 4, 5, 6
flag_meanings : surface_not_detected no_surface_wind
no_scattering_layer_found no_top_layer_found none_little
weak moderate moderate_high high very_high
"""

In [ ]:
gtl1_df['bsnow_conf'].value_counts()

In [ ]:
gtl1_df['lon'][0],gtl1_df['lat'][0]